In [20]:
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)

from client import Client
import pandas as pd
from supported_modles import Supported_modles
import utils
from fedavg import Fedavg
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import random

In [21]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [22]:
clients = utils.set_data(True)

Creating node1.
Creating node2.
Creating node3.
Creating node4.
Creating node5.
DOWNSAMLING node3


In [23]:
selected_model = Supported_modles.logistic_regression

In [24]:
f1_before = []
for client in clients:
    # client.downsample_data(['Destination Port','Flow Duration','Flow Bytes/s'])
    client.init_empty_model(selected_model,0.01,None)
    client.split_data()
    client.prep_data()
    client.train_model(selected_model)
    f1_before.append(client.f1)

In [25]:
f1_before

[0.9987980378780495,
 0.9981154570978039,
 0.9762174405436014,
 0.9981196663613134,
 0.998695591097911]

In [30]:
selected_model = Supported_modles.SGD_classifier

In [31]:
# clients = utils.set_data(True)
for client in clients:
    # client.split_data()
    client.init_empty_model(selected_model,0.001,None)

In [32]:
fedavg = Fedavg("global", 0.001)
fedavg.init_global_model(selected_model, None,78)

[Errno 98] Address already in use
Waitiing for a Connection..


In [33]:
number_of_rounds =50
batch_size = 0.2
epochs = 20
prep = StandardScaler() 


for _ in range(number_of_rounds):
    print(f'Starting new round!')

    applicable_clients = random.sample((clients), random.randint(2, 4))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0
    

    for client in applicable_clients:
        # print(f'Client name: {client.name}')

        X = client.x
        y = client.y

        X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=batch_size, stratify=y, random_state=random.randint(0,10))
       
        # X_train = prep.fit_transform(X_train)

        dataset_size += X_train.shape[0]
        sample_weights = compute_sample_weight('balanced', y=y_train)

        fedavg.load_global_model(client.model, selected_model)
        fedavg.train_local_agent(X_train, y_train, client.model, epochs, sample_weights, selected_model)
        round_weights.append(X_train.shape[0])
        applicable_models.append(client.model)


    round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
    # round_weights = weights
    fedavg.update_global_model(applicable_models, round_weights, selected_model)
    print(fedavg.model.intercept_)

Starting new round!
[-1.42460015]
Starting new round!
[-1.13735378]
Starting new round!
[-1.49025118]
Starting new round!
[-2.00654842]
Starting new round!
[-1.46212165]
Starting new round!
[-1.38347927]
Starting new round!
[-1.28266448]
Starting new round!
[-1.9356928]
Starting new round!
[-0.47679585]
Starting new round!
[-1.51380943]
Starting new round!
[-0.67098428]
Starting new round!
[-0.37208545]
Starting new round!
[-1.03548755]
Starting new round!
[-1.84481393]
Starting new round!
[-1.30826009]
Starting new round!
[-1.39318034]
Starting new round!
[-1.27555453]
Starting new round!
[-1.35986574]
Starting new round!
[-1.42960143]
Starting new round!
[-1.39527299]
Starting new round!
[-0.84421742]
Starting new round!
[-1.51724269]
Starting new round!
[-2.08898618]
Starting new round!
[-1.20098129]
Starting new round!
[-0.94560117]
Starting new round!
[-1.20586156]
Starting new round!
[-1.43387917]
Starting new round!
[-0.89930606]
Starting new round!
[-1.65562261]
Starting new ro

In [34]:
f1_fedavg = []
for client in clients:
    # x = prep.transform(client.x_test)
    x = client.x_test
    y_hat = fedavg.model.predict(x)
    f1_fedavg.append(f1_score(client.y_test,y_hat))
print(f1_fedavg)

[0.9720025510204081, 0.936849710982659, 0.7098445595854923, 0.8967811622380644, 0.9954929577464788]


In [35]:
i = 0
for client in clients:
    print(f'Difference {f1_fedavg[i]-f1_before[i]}')
    i += 1

Difference -0.026795486857641437
Difference -0.0612657461151449
Difference -0.26637288095810907
Difference -0.10133850412324907
Difference -0.003202633351432116
